In [35]:
import pandas as pd

# JSON 파일을 데이터프레임으로 읽기
df1 = pd.read_json("D:\\문제\\bigdata7-final_project--1\\결과\\2019 고3 3월 국어.json")
df2 = pd.read_json("D:\\문제\\bigdata7-final_project--1\\해설(완)\\국어\\고3\\2019 고3 3월 국어 해설.json")

# df1에서 multiple_answer 컬럼만 선택
df1

ValueError: Trailing data

In [15]:
import pandas as pd
import json
# JSON 파일을 줄 단위로 읽고 데이터프레임으로 변환하는 함수
def read_json_lines(file_path):
    data = []
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            data.append(json.loads(line))
    return pd.json_normalize(data)

# JSON 파일을 데이터프레임으로 불러오기
df1 = read_json_lines("D:\\문제\\bigdata7-final_project--1\\해설(완)\\국어\\고3\\2019 고3 3월 국어 해설.json")
df2 = read_json_lines("D:\\문제\\bigdata7-final_project--1\\정답(완)\\국어\\고3\\2019 고3 3월 국어 해설.json")

JSONDecodeError: Expecting property name enclosed in double quotes: line 2 column 1 (char 2)

In [40]:
import pandas as pd
import glob
import os
import json

def load_json_files_to_dataframe(directory):
    all_files = glob.glob(os.path.join(directory, "*.json"))
    df_dict = {}

    for filename in all_files:
        try:
            with open(filename, 'r', encoding='utf-8') as f:
                data = json.load(f)
                if not data:  # 빈 파일 체크
                    print(f"빈 파일 건너뜀: {filename}")
                    continue
                df = pd.json_normalize(data)
                file_key = ' '.join(filename.split(os.path.sep)[-1].split(' ')[:4])
                if file_key in df_dict:
                    df_dict[file_key] = pd.concat([df_dict[file_key], df], ignore_index=True)
                else:
                    df_dict[file_key] = df
        except json.JSONDecodeError:
            print(f"JSON 디코딩 오류로 인해 파일 건너뜀: {filename}")
            continue
    
    return df_dict

def combine_dataframes_from_directories(dir1, dir2):
    df_dict1 = load_json_files_to_dataframe(dir1)
    df_dict2 = load_json_files_to_dataframe(dir2)
    
    combined_df_dict = {}

    for key in df_dict1.keys():
        if key in df_dict2:
            print(f"Processing key: {key}")
            print(f"Columns in df_dict1[{key}]: {df_dict1[key].columns}")
            print(f"Columns in df_dict2[{key}]: {df_dict2[key].columns}")

            # Ensure the required columns are present
            if 'question_num' in df_dict1[key].columns and 'question_num' in df_dict2[key].columns:
                df2_filtered = df_dict2[key][['question_num', 'multiple_answer']]
                combined_df = pd.merge(df_dict1[key], df2_filtered, on='question_num', how='left')
                combined_df_dict[key] = combined_df
            else:
                print(f"Key {key} does not have the required 'question_num' column.")
                combined_df_dict[key] = df_dict1[key]
        else:
            print(f"Key {key} not found in both directories.")
            combined_df_dict[key] = df_dict1[key]

    return combined_df_dict

def save_dataframes_to_json(combined_df_dict, output_directory):
    if not os.path.exists(output_directory):
        os.makedirs(output_directory)

    for key, df in combined_df_dict.items():
        output_path = os.path.join(output_directory, f"{key}.json")
        json_data = df.to_dict(orient='records')
        with open(output_path, 'w', encoding='utf-8') as f:
            json.dump(json_data, f, ensure_ascii=False, indent=4)
        print(f"JSON 파일로 저장 완료: {output_path}")

# 디렉토리 경로 지정
input_directory_1 = "D:\\문제\\bigdata7-final_project--1\\해설(완)\\국어\\고3"
input_directory_2 = "D:\\문제\\bigdata7-final_project--1\\정답(완)\\국어\\고3"
output_directory = "D:\\문제\\bigdata7-final_project--1\\결과"

# 데이터프레임 결합 및 저장
combined_df_dict = combine_dataframes_from_directories(input_directory_1, input_directory_2)
save_dataframes_to_json(combined_df_dict, output_directory)


빈 파일 건너뜀: D:\문제\bigdata7-final_project--1\해설(완)\국어\고3\2022 고3 3월 언매 해설.json
빈 파일 건너뜀: D:\문제\bigdata7-final_project--1\해설(완)\국어\고3\2022 고3 3월 화작 해설.json
Processing key: 2019 고3 10월 국어
Columns in df_dict1[2019 고3 10월 국어]: Index(['question_num', 'text_exp', 'question_exp'], dtype='object')
Columns in df_dict2[2019 고3 10월 국어]: Index(['subject_cat', 'question_num', 'multiple_answer'], dtype='object')
Processing key: 2019 고3 3월 국어
Columns in df_dict1[2019 고3 3월 국어]: Index(['question_num', 'text_exp', 'question_exp'], dtype='object')
Columns in df_dict2[2019 고3 3월 국어]: Index(['subject_cat', 'question_num', 'multiple_answer'], dtype='object')
Processing key: 2019 고3 4월 국어
Columns in df_dict1[2019 고3 4월 국어]: Index(['question_num', 'text_exp', 'question_exp'], dtype='object')
Columns in df_dict2[2019 고3 4월 국어]: Index(['subject_cat', 'question_num', 'multiple_answer'], dtype='object')
Processing key: 2019 고3 7월 국어
Columns in df_dict1[2019 고3 7월 국어]: Index(['question_num', 'text_exp', 'question_ex

In [41]:
import pandas as pd
import glob
import os
import json

def load_json_files_to_dataframe(directory):
    all_files = glob.glob(os.path.join(directory, "*.json"))
    df_dict = {}

    for filename in all_files:
        try:
            with open(filename, 'r', encoding='utf-8') as f:
                data = json.load(f)
                if not data:  # 빈 파일 체크
                    print(f"빈 파일 건너뜀: {filename}")
                    continue
                df = pd.json_normalize(data)
                file_key = ' '.join(filename.split(os.path.sep)[-1].split(' ')[:4])
                if file_key in df_dict:
                    df_dict[file_key] = pd.concat([df_dict[file_key], df], ignore_index=True)
                else:
                    df_dict[file_key] = df
        except json.JSONDecodeError:
            print(f"JSON 디코딩 오류로 인해 파일 건너뜀: {filename}")
            continue
    
    return df_dict

def combine_dataframes_from_directories(dir1, dir2):
    df_dict1 = load_json_files_to_dataframe(dir1)
    df_dict2 = load_json_files_to_dataframe(dir2)
    
    combined_df_dict = {}

    for key in df_dict1.keys():
        if key in df_dict2:
            print(f"Processing key: {key}")
            print(f"Columns in df_dict1[{key}]: {df_dict1[key].columns}")
            print(f"Columns in df_dict2[{key}]: {df_dict2[key].columns}")

            # Ensure the required columns are present
            if 'question_num' in df_dict1[key].columns and 'question_num' in df_dict2[key].columns:
                df2_filtered = df_dict2[key][['question_num',"text_exp","question_exp" ,'multiple_answer']]
                combined_df = pd.merge(df_dict1[key], df2_filtered, on='question_num', how='left')
                combined_df_dict[key] = combined_df
            else:
                print(f"Key {key} does not have the required 'question_num' column.")
                combined_df_dict[key] = df_dict1[key]
        else:
            print(f"Key {key} not found in both directories.")
            combined_df_dict[key] = df_dict1[key]

    return combined_df_dict

def save_dataframes_to_json(combined_df_dict, output_directory):
    if not os.path.exists(output_directory):
        os.makedirs(output_directory)

    for key, df in combined_df_dict.items():
        output_path = os.path.join(output_directory, f"{key}.json")
        json_data = df.to_dict(orient='records')
        with open(output_path, 'w', encoding='utf-8') as f:
            json.dump(json_data, f, ensure_ascii=False, indent=4)
        print(f"JSON 파일로 저장 완료: {output_path}")

# 디렉토리 경로 지정
input_directory_1 = "D:\\문제\\bigdata7-final_project--1\\문제(완)\\국어\\고3"
input_directory_2 = "D:\\문제\\bigdata7-final_project--1\\결과"
output_directory = "D:\\문제\\bigdata7-final_project--1\\결과(최종)"

# 데이터프레임 결합 및 저장
combined_df_dict = combine_dataframes_from_directories(input_directory_1, input_directory_2)
save_dataframes_to_json(combined_df_dict, output_directory)

빈 파일 건너뜀: D:\문제\bigdata7-final_project--1\문제(완)\국어\고3\2020 고3 수능 국어 해설.json
JSON 디코딩 오류로 인해 파일 건너뜀: D:\문제\bigdata7-final_project--1\결과\2022 고3 3월 언매.json
JSON 디코딩 오류로 인해 파일 건너뜀: D:\문제\bigdata7-final_project--1\결과\2022 고3 3월 화작.json
Processing key: 2019 고3 10월 국어.json
Columns in df_dict1[2019 고3 10월 국어.json]: Index(['grade', 'yyyy', 'mm', 'host', 'subject_cat', 'question_num', 'points',
       'text_title', 'text', 'text_yn', 'question', 'paragraph', 'choice1',
       'choice2', 'choice3', 'choice4', 'choice5', 'multiple_answer',
       'short_answer'],
      dtype='object')
Columns in df_dict2[2019 고3 10월 국어.json]: Index(['question_num', 'text_exp', 'question_exp', 'multiple_answer'], dtype='object')
Processing key: 2019 고3 3월 국어.json
Columns in df_dict1[2019 고3 3월 국어.json]: Index(['grade', 'yyyy', 'mm', 'host', 'subject_cat', 'question_num', 'points',
       'text_title', 'text', 'text_yn', 'question', 'paragraph', 'choice1',
       'choice2', 'choice3', 'choice4', 'choice5', 'multipl

In [59]:
import os
import fitz  # PyMuPDF
import pytesseract
import json
import glob
import re

# Tesseract 경로 설정 (필요시)
tesseract_cmd = "C:/Program Files/Tesseract-OCR/tesseract.exe"
pytesseract.pytesseract.tesseract_cmd = tesseract_cmd

def clean_preamble(text):
    text = re.sub(r'이 문제지에 관한 저작권은 한국교육과정평가원에 있습니다', '', text)
    text = re.sub(r'\n', '', text)
    text = re.sub(r'', '', text)
    return text

def pdf_to_json(pdf_path, output_dir):
    # 파일명에서 메타데이터 추출
    filename = os.path.basename(pdf_path)
    name_parts = filename.split(' ')
    
    # 파일명 형식 확인
    if len(name_parts) < 4:
        print(f"파일명 형식 오류: {filename}")
        return
    
    year = name_parts[0]
    target = name_parts[1]
    purpose = name_parts[2]
    subject = name_parts[3].split('.')[0]
    
    doc = fitz.open(pdf_path)

    full_text = ""

    for page_num in range(len(doc)):
        page = doc.load_page(page_num)
        text = page.get_text("text")
        images = page.get_images(full=True)

        # 이미지 추출 및 OCR 수행
        for img_index, img in enumerate(images):
            xref = img[0]
            base_image = doc.extract_image(xref)
            image_bytes = base_image["image"]
            image_filename = f'{filename}_page{page_num+1}_img{img_index+1}.png'
            image_path = os.path.join(output_dir, 'images', filename.split('.')[0], image_filename)
            
            # 이미지 파일로 저장
            os.makedirs(os.path.dirname(image_path), exist_ok=True)
            with open(image_path, 'wb') as img_file:
                img_file.write(image_bytes)
            
            # 이미지에서 OCR 수행
            img_text = pytesseract.image_to_string(image_path, lang="kor+eng")
            text += img_text

        full_text += text + "\n"

    final_data = {
        "년도": year,
        "대상": target,
        "목적": purpose,
        "과목": subject,
        "text": clean_preamble(full_text.strip())
    }

    # JSON 파일 저장
    output_json = os.path.join(output_dir, f'{filename.split(".")[0]}.json')
    with open(output_json, 'w', encoding='utf-8') as f:
        json.dump(final_data, f, ensure_ascii=False, indent=4)

    print(f"Converted {pdf_path} to {output_json}")

def convert_all_pdfs_in_directory(input_directory, output_directory):
    pdf_files = glob.glob(os.path.join(input_directory, "*.pdf"))
    
    if not os.path.exists(output_directory):
        os.makedirs(output_directory)
    
    for pdf_file in pdf_files:
        pdf_to_json(pdf_file, output_directory)

# 입력 디렉토리 경로 설정
input_directory = "D:\\문제\\bigdata7-final_project--1\\문제\\국어\\고1"

# 출력 디렉토리 경로 설정
output_directory = "D:\\문제\\bigdata7-final_project--1\\문제(통텍스트)\\국어\\고1"

# 모든 PDF 파일 변환
convert_all_pdfs_in_directory(input_directory, output_directory)


Converted D:\문제\bigdata7-final_project--1\문제\국어\고1\2015 고1 11월 국어.pdf to D:\문제\bigdata7-final_project--1\문제(통텍스트)\국어\고1\2015 고1 11월 국어.json
Converted D:\문제\bigdata7-final_project--1\문제\국어\고1\2015 고1 3월 국어.pdf to D:\문제\bigdata7-final_project--1\문제(통텍스트)\국어\고1\2015 고1 3월 국어.json
Converted D:\문제\bigdata7-final_project--1\문제\국어\고1\2015 고1 6월 국어.pdf to D:\문제\bigdata7-final_project--1\문제(통텍스트)\국어\고1\2015 고1 6월 국어.json
Converted D:\문제\bigdata7-final_project--1\문제\국어\고1\2015 고1 9월 국어.pdf to D:\문제\bigdata7-final_project--1\문제(통텍스트)\국어\고1\2015 고1 9월 국어.json
Converted D:\문제\bigdata7-final_project--1\문제\국어\고1\2016 고1 11월 국어.pdf to D:\문제\bigdata7-final_project--1\문제(통텍스트)\국어\고1\2016 고1 11월 국어.json
Converted D:\문제\bigdata7-final_project--1\문제\국어\고1\2016 고1 3월 국어.pdf to D:\문제\bigdata7-final_project--1\문제(통텍스트)\국어\고1\2016 고1 3월 국어.json
Converted D:\문제\bigdata7-final_project--1\문제\국어\고1\2016 고1 6월 국어.pdf to D:\문제\bigdata7-final_project--1\문제(통텍스트)\국어\고1\2016 고1 6월 국어.json
Converted D:\문제\bigdata7-final